In [1]:
!which python

/opt/conda/bin/python


In [2]:
# !python

In [3]:
# from https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283/code

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

"""
This simple scripts demonstrates the use of xgboost eval results to get the best round
for the current fold and accross folds. 
It also shows an upsampling method that limits cross-validation overfitting.
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import gc
from numba import jit
from sklearn.preprocessing import LabelEncoder
import time 
from datetime import datetime
from tqdm import tqdm

In [5]:
@jit
def eval_gini(y_true, y_prob):
    """
    Original author CPMP : https://www.kaggle.com/cpmpml
    In kernel : https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))



In [6]:
def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


In [7]:
gc.enable()

trn_df = pd.read_csv("../data/train.csv", index_col=0)
sub_df = pd.read_csv("../data/test.csv", index_col=0)

target = trn_df["target"]
del trn_df["target"]

In [8]:
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
    "ps_reg_03",  #            : 1408.42 / shadow  511.15
    "ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
    "ps_ind_03",  #            : 1219.47 / shadow  230.55
    "ps_ind_15",  #            :  922.18 / shadow  242.00
    "ps_reg_02",  #            :  920.65 / shadow  267.50
    "ps_car_14",  #            :  798.48 / shadow  549.58
    "ps_car_12",  #            :  731.93 / shadow  293.62
    "ps_car_01_cat",  #        :  698.07 / shadow  178.72
    "ps_car_07_cat",  #        :  694.53 / shadow   36.35
    "ps_ind_17_bin",  #        :  620.77 / shadow   23.15
    "ps_car_03_cat",  #        :  611.73 / shadow   50.67
    "ps_reg_01",  #            :  598.60 / shadow  178.57
    "ps_car_15",  #            :  593.35 / shadow  226.43
    "ps_ind_01",  #            :  547.32 / shadow  154.58
    "ps_ind_16_bin",  #        :  475.37 / shadow   34.17
    "ps_ind_07_bin",  #        :  435.28 / shadow   28.92
    "ps_car_06_cat",  #        :  398.02 / shadow  212.43
    "ps_car_04_cat",  #        :  376.87 / shadow   76.98
    "ps_ind_06_bin",  #        :  370.97 / shadow   36.13
    "ps_car_09_cat",  #        :  214.12 / shadow   81.38
    "ps_car_02_cat",  #        :  203.03 / shadow   26.67
    "ps_ind_02_cat",  #        :  189.47 / shadow   65.68
    "ps_car_11",  #            :  173.28 / shadow   76.45
    "ps_car_05_cat",  #        :  172.75 / shadow   62.92
    "ps_calc_09",  #           :  169.13 / shadow  129.72
    "ps_calc_05",  #           :  148.83 / shadow  120.68
    "ps_ind_08_bin",  #        :  140.73 / shadow   27.63
    "ps_car_08_cat",  #        :  120.87 / shadow   28.82
    "ps_ind_09_bin",  #        :  113.92 / shadow   27.05
    "ps_ind_04_cat",  #        :  107.27 / shadow   37.43
    "ps_ind_18_bin",  #        :   77.42 / shadow   25.97
    "ps_ind_12_bin",  #        :   39.67 / shadow   15.52
    "ps_ind_14",  #            :   37.37 / shadow   16.65
    "ps_car_11_cat" # Very nice spot from Tilii : https://www.kaggle.com/tilii7
]
# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [9]:
f_cats = [f for f in train_features if "_cat" in f]

In [10]:
f_cars = [f for f in train_features if "_car" in f and '_cat' not in f]

In [11]:
f_cars

['ps_car_13', 'ps_car_14', 'ps_car_12', 'ps_car_15', 'ps_car_11']

In [12]:
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    trn_df[name1] = trn_df[f1].apply(lambda x: str(x)) + "_" + trn_df[f2].apply(lambda x: str(x))
    sub_df[name1] = sub_df[f1].apply(lambda x: str(x)) + "_" + sub_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(trn_df[name1].values) + list(sub_df[name1].values))
    trn_df[name1] = lbl.transform(list(trn_df[name1].values))
    sub_df[name1] = lbl.transform(list(sub_df[name1].values))

    train_features.append(name1)

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [13]:
f_cats

['ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_07_cat',
 'ps_car_03_cat',
 'ps_car_06_cat',
 'ps_car_04_cat',
 'ps_car_09_cat',
 'ps_car_02_cat',
 'ps_ind_02_cat',
 'ps_car_05_cat',
 'ps_car_08_cat',
 'ps_ind_04_cat',
 'ps_car_11_cat']

In [12]:
col_calc_mean = ['ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15']

In [14]:
for f in f_cats:
    for col in f_cars:

        new_col1 = '{}_{}_mean'.format(col, f) 
        new_col2 = '{}_{}_median'.format(col, f) 
        new_col3 = '{}_{}_skew'.format(col, f) 
        new_col4 = '{}_{}_kurtosis'.format(col, f) 
        trn_df[new_col1] = 0
        trn_df[new_col2] = 0
        trn_df[new_col3] = 0
        trn_df[new_col4] = 0
        
        sub_df[new_col1] = 0
        sub_df[new_col2] = 0
        sub_df[new_col3] = 0
        sub_df[new_col4] = 0

In [17]:
-

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 5/5 [01:27<00:00, 18.26s/it]


In [21]:
trn_df = trn_df[train_features]
sub_df = sub_df[train_features]


In [22]:
pd.set_option('display.max_columns', None)

In [23]:
trn_df.head()

,ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,ps_ind_17_bin,ps_car_03_cat,ps_reg_01,ps_car_15,ps_ind_01,ps_ind_16_bin,ps_ind_07_bin,ps_car_06_cat,ps_car_04_cat,ps_ind_06_bin,ps_car_09_cat,ps_car_02_cat,ps_ind_02_cat,ps_car_11,ps_car_05_cat,ps_calc_09,ps_calc_05,ps_ind_08_bin,ps_car_08_cat,ps_ind_09_bin,ps_ind_04_cat,ps_ind_18_bin,ps_ind_12_bin,ps_ind_14,ps_car_11_cat,ps_reg_01_plus_ps_car_02_cat,ps_reg_01_plus_ps_car_04_cat,ps_car_13_ps_ind_05_cat_mean,ps_car_13_ps_ind_05_cat_median,ps_car_13_ps_car_01_cat_mean,ps_car_13_ps_car_01_cat_median,ps_car_13_ps_car_07_cat_mean,ps_car_13_ps_car_07_cat_median,ps_car_13_ps_car_03_cat_mean,ps_car_13_ps_car_03_cat_median,ps_car_13_ps_car_06_cat_mean,ps_car_13_ps_car_06_cat_median,ps_car_13_ps_car_04_cat_mean,ps_car_13_ps_car_04_cat_median,ps_car_13_ps_car_09_cat_mean,ps_car_13_ps_car_09_cat_median,ps_car_13_ps_car_02_cat_mean,ps_car_13_ps_car_02_cat_median,ps_car_13_ps_ind_02_cat_mean,ps_car_13_ps_ind_02_cat_median,ps_car_13_ps_car_05_cat_mean,ps_car_13_ps_car_05_cat_median,ps_car_13_ps_car_08_cat_mean,ps_car_13_ps_car_08_cat_median,ps_car_13_ps_ind_04_cat_mean,ps_car_13_ps_ind_04_cat_median,ps_car_13_ps_car_11_cat_mean,ps_car_13_ps_car_11_cat_median,ps_car_14_ps_ind_05_cat_mean,ps_car_14_ps_ind_05_cat_median,ps_car_14_ps_car_01_cat_mean,ps_car_14_ps_car_01_cat_median,ps_car_14_ps_car_07_cat_mean,ps_car_14_ps_car_07_cat_median,ps_car_14_ps_car_03_cat_mean,ps_car_14_ps_car_03_cat_median,ps_car_14_ps_car_06_cat_mean,ps_car_14_ps_car_06_cat_median,ps_car_14_ps_car_04_cat_mean,ps_car_14_ps_car_04_cat_median,ps_car_14_ps_car_09_cat_mean,ps_car_14_ps_car_09_cat_median,ps_car_14_ps_car_02_cat_mean,ps_car_14_ps_car_02_cat_median,ps_car_14_ps_ind_02_cat_mean,ps_car_14_ps_ind_02_cat_median,ps_car_14_ps_car_05_cat_mean,ps_car_14_ps_car_05_cat_median,ps_car_14_ps_car_08_cat_mean,ps_car_14_ps_car_08_cat_median,ps_car_14_ps_ind_04_cat_mean,ps_car_14_ps_ind_04_cat_median,ps_car_14_ps_car_11_cat_mean,ps_car_14_ps_car_11_cat_median,ps_car_12_ps_ind_05_cat_mean,ps_car_12_ps_ind_05_cat_median,ps_car_12_ps_car_01_cat_mean,ps_car_12_ps_car_01_cat_median,ps_car_12_ps_car_07_cat_mean,ps_car_12_ps_car_07_cat_median,ps_car_12_ps_car_03_cat_mean,ps_car_12_ps_car_03_cat_median,ps_car_12_ps_car_06_cat_mean,ps_car_12_ps_car_06_cat_median,ps_car_12_ps_car_04_cat_mean,ps_car_12_ps_car_04_cat_median,ps_car_12_ps_car_09_cat_mean,ps_car_12_ps_car_09_cat_median,ps_car_12_ps_car_02_cat_mean,ps_car_12_ps_car_02_cat_median,ps_car_12_ps_ind_02_cat_mean,ps_car_12_ps_ind_02_cat_median,ps_car_12_ps_car_05_cat_mean,ps_car_12_ps_car_05_cat_median,ps_car_12_ps_car_08_cat_mean,ps_car_12_ps_car_08_cat_median,ps_car_12_ps_ind_04_cat_mean,ps_car_12_ps_ind_04_cat_median,ps_car_12_ps_car_11_cat_mean,ps_car_12_ps_car_11_cat_median,ps_car_15_ps_ind_05_cat_mean,ps_car_15_ps_ind_05_cat_median,ps_car_15_ps_car_01_cat_mean,ps_car_15_ps_car_01_cat_median,ps_car_15_ps_car_07_cat_mean,ps_car_15_ps_car_07_cat_median,ps_car_15_ps_car_03_cat_mean,ps_car_15_ps_car_03_cat_median,ps_car_15_ps_car_06_cat_mean,ps_car_15_ps_car_06_cat_median,ps_car_15_ps_car_04_cat_mean,ps_car_15_ps_car_04_cat_median,ps_car_15_ps_car_09_cat_mean,ps_car_15_ps_car_09_cat_median,ps_car_15_ps_car_02_cat_mean,ps_car_15_ps_car_02_cat_median,ps_car_15_ps_ind_02_cat_mean,ps_car_15_ps_ind_02_cat_median,ps_car_15_ps_car_05_cat_mean,ps_car_15_ps_car_05_cat_median,ps_car_15_ps_car_08_cat_mean,ps_car_15_ps_car_08_cat_median,ps_car_15_ps_ind_04_cat_mean,ps_car_15_ps_ind_04_cat_median,ps_car_15_ps_car_11_cat_mean,ps_car_15_ps_car_11_cat_median,ps_car_11_ps_ind_05_cat_mean,ps_car_11_ps_ind_05_cat_median,ps_car_11_ps_car_01_cat_mean,ps_car_11_ps_car_01_cat_median,ps_car_11_ps_car_07_cat_mean,ps_car_11_ps_car_07_cat_median,ps_car_11_ps_car_03_cat_mean,ps_car_11_ps_car_03_cat_median,ps_car_11_ps_car_06_cat_mean,ps_car_11_ps_car_06_cat_median,ps_car_11_ps_car_04_cat_mean,ps_car_11_ps_car_04_cat_median,ps_car_11_ps_car_09_cat_m

In [24]:
f_cats = [f for f in trn_df.columns if "_cat" in f]

In [25]:
for f in f_cats:
    trn_df[f + "_avg"], sub_df[f + "_avg"] = target_encode(trn_series=trn_df[f],
                                         tst_series=sub_df[f],
                                         target=target,
                                         min_samples_leaf=200,
                                         smoothing=10,
                                         noise_level=0)

In [26]:
n_splits = 5
n_estimators = 200
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=15) 
imp_df = np.zeros((len(trn_df.columns), n_splits))
xgb_evals = np.zeros((n_estimators, n_splits))
oof = np.empty(len(trn_df))
sub_preds = np.zeros(len(sub_df))
increase = True
np.random.seed(0)


In [27]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
    trn_dat, trn_tgt = trn_df.iloc[trn_idx], target.iloc[trn_idx]
    val_dat, val_tgt = trn_df.iloc[val_idx], target.iloc[val_idx]

    print(sum(trn_tgt==0)/sum(trn_tgt==1))
    
    sp = sum(trn_tgt==0)/sum(trn_tgt==1)
    clf = XGBClassifier(n_estimators=n_estimators,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.8, 
                        colsample_bytree=.8,
                        scale_pos_weight=sp,
                        gamma=1,
                        reg_alpha=0,
                        reg_lambda=1,
                        nthread=6)
    # Upsample during cross validation to avoid having the same samples
    # in both train and validation sets
    # Validation set is not up-sampled to monitor overfitting
    if increase:
        # Get positive examples
        pos = pd.Series(trn_tgt == 1)
        # Add positive examples
        trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
        trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
        # Shuffle data
        idx = np.arange(len(trn_dat))
        np.random.shuffle(idx)
        trn_dat = trn_dat.iloc[idx]
        trn_tgt = trn_tgt.iloc[idx]
        
    clf.fit(trn_dat, trn_tgt, 
            eval_set=[(trn_dat, trn_tgt), (val_dat, val_tgt)],
            eval_metric=gini_xgb,
            early_stopping_rounds=None,
            verbose=False)
            
    # Keep feature importances
    imp_df[:, fold_] = clf.feature_importances_

    # Find best round for validation set
    xgb_evals[:, fold_] = clf.evals_result_["validation_1"]["gini"]
    # Xgboost provides best round starting from 0 so it has to be incremented
    best_round = np.argsort(xgb_evals[:, fold_])[::-1][0]
    print(best_round)
    
    # Predict OOF and submission probas with the best round
    oof[val_idx] = clf.predict_proba(val_dat, ntree_limit=int(best_round))[:, 1]
    # Update submission
    sub_preds += clf.predict_proba(sub_df, ntree_limit=int(best_round))[:, 1] / n_splits

    # Display results
    print("Fold %2d : %.6f @%4d / best score is %.6f @%4d"
          % (fold_ + 1,
             eval_gini(val_tgt, oof[val_idx]),
             n_estimators,
             xgb_evals[best_round, fold_],
             best_round))
          


26.4369922213
91
Fold  1 : 0.271929 @ 200 / best score is 0.272182 @  91
26.4369922213
111
Fold  2 : 0.278302 @ 200 / best score is 0.278628 @ 111
26.4369922213
81
Fold  3 : 0.301982 @ 200 / best score is 0.302092 @  81
26.4370498415
101
Fold  4 : 0.272883 @ 200 / best score is 0.273043 @ 101
26.435526619
67
Fold  5 : 0.271686 @ 200 / best score is 0.271827 @  67


In [28]:
print("Full OOF score : %.6f" % eval_gini(target, oof))
# org with clipping, Full OOF score : 0.284952, LB: 0.275
# org no clipping, Full OOF score : 0.284952, LB: 0.275
# org, removed ntree_limit, kaggle/python: Full OOF score : 0.283630, LB: 0.274
# org, with ntree_limit, kaggle/python: Full OOF score : 0.284745, LB: 0.282
# org + counts for 4, Full OOF score : 0.279249
# org + counts for all cars, only mean: 0.279233

Full OOF score : 0.279233


In [ ]:
# Compute mean score and std
mean_eval = np.mean(xgb_evals, axis=1)
std_eval = np.std(xgb_evals, axis=1)
best_round = np.argsort(mean_eval)[::-1][0]

print("Best mean score : %.6f + %.6f @%4d"
      % (mean_eval[best_round], std_eval[best_round], best_round))
    


In [ ]:
importances = sorted([(trn_df.columns[i], imp) for i, imp in enumerate(imp_df.mean(axis=1))],
                     key=lambda x: x[1])

for f, imp in importances[::-1]:
    print("%-34s : %10.4f" % (f, imp))

In [ ]:
sub_df["target"] = sub_preds
now = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
fn = '../submissions/sub.xgb.{}GMT'.format(now)
sub_df[["target"]].to_csv(fn, index=True, float_format="%.9f")

In [ ]:
print(now)

In [ ]:
sub_df.tail()